<a href="https://colab.research.google.com/github/Baekhyeonjeong/movie-review-/blob/main/%EC%98%81%ED%99%94_%EB%B9%84%EA%B8%B4%EC%96%B4%EA%B2%8C%EC%9D%B8_%EB%A6%AC%EB%B7%B0_%ED%98%95%ED%83%9C%EC%86%8C_%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 환경 준비

In [94]:
# konlpy 설치하기
!pip install konlpy

In [95]:
# konlpy를 실행하기
import konlpy

In [96]:
## pandas을 실행하기, 단 실행 별명을 pd로
import pandas as pd

# 데이터 불러오기

In [97]:
# 구글 드라이브 연결을 위한 기본 세팅 
!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [98]:
## 문서 ID로 실제 파일 불러오기
# 비긴.csv
## https://drive.google.com/file/d/1n7Lw9qcHqwaxNsqJuYffodET9MNJkA2d/view?usp=sharing


rawdata_downloaded = drive.CreateFile({'id': '1n7Lw9qcHqwaxNsqJuYffodET9MNJkA2d'})
rawdata_downloaded.GetContentFile('비긴.csv')

In [99]:
# "rawdata.txt" 파일의 내용을 "원본데이터" 변수로 불러오기
원본데이터 = pd.read_csv('비긴.csv', sep=",")

In [100]:
# "원본데이터" 변수 내용 확인하기
원본데이터

,Unnamed: 0,nickname,text,score,like,dislike,time,watch_movie
0,0,zoo keeper(kimc****),박평식평론가는 그렇게 재밌는영화가 없으면 영화를 무슨재미로 볼까? ㅉ,10,6398,835,2014-08-13 09:04:00,0
1,1,samm****,평점 처음써봐요...진짜 오에스티며 스토리며 영상미며....최고.. .원스를능가한다...,10,3359,370,2014-08-13 10:05:00,0
2,2,반우치(dc71****),일단 노래가 엄청좋다.,10,2917,172,2014-08-13 00:20:00,0
3,3,라루스(lpl2****),영화가 끝나고나서 작은 락페스티벌에 다녀온 느낌이 들었다,10,2625,189,2014-08-13 08:34:00,0
4,4,타자왕(ieew****),원스가 모든 면에서 진화한 느낌. 노래 나올 때 들썩들썩하면서 보았음.,10,2112,237,2014-08-13 02:08:00,0
...,...,...,...,...,...,...,...,...
16895,16895,갈구(usfy****),여주 개 옷 생김 .덧니도 짜증 발음도 짜증,1,4,18,2016-07-09 02:05:00,0
16896,16896,여행(rhtj****),무한도전이 이영화 를 망쳤다먼 생각으로 이영화를 더빙을 한거지?이게 어울린다고 생각...,1,7,21,2015-09-27 22:56:00,0
16897,16897,판도라상자(igno****),무한도전 쓰레기들로 인해 최악의 더빙 영화가 되었다 장난치려면 다른 쓰레기 영화 장...,1,13,27,2015-09-29 23:36:00,0
16898,16898,sung****,노래가 너무 좋았고 다 괜찮았음,1,1,18,2014-09-05 09:25:00,관람객


In [102]:
원본데이터NAN삭제 = 원본데이터.dropna(axis=0)
## 바로: NAN값(빈값)을 삭제하기 위한 프로세스

In [103]:
원본데이터NAN삭제

,Unnamed: 0,nickname,text,score,like,dislike,time,watch_movie
0,0,zoo keeper(kimc****),박평식평론가는 그렇게 재밌는영화가 없으면 영화를 무슨재미로 볼까? ㅉ,10,6398,835,2014-08-13 09:04:00,0
1,1,samm****,평점 처음써봐요...진짜 오에스티며 스토리며 영상미며....최고.. .원스를능가한다...,10,3359,370,2014-08-13 10:05:00,0
2,2,반우치(dc71****),일단 노래가 엄청좋다.,10,2917,172,2014-08-13 00:20:00,0
3,3,라루스(lpl2****),영화가 끝나고나서 작은 락페스티벌에 다녀온 느낌이 들었다,10,2625,189,2014-08-13 08:34:00,0
4,4,타자왕(ieew****),원스가 모든 면에서 진화한 느낌. 노래 나올 때 들썩들썩하면서 보았음.,10,2112,237,2014-08-13 02:08:00,0
...,...,...,...,...,...,...,...,...
16895,16895,갈구(usfy****),여주 개 옷 생김 .덧니도 짜증 발음도 짜증,1,4,18,2016-07-09 02:05:00,0
16896,16896,여행(rhtj****),무한도전이 이영화 를 망쳤다먼 생각으로 이영화를 더빙을 한거지?이게 어울린다고 생각...,1,7,21,2015-09-27 22:56:00,0
16897,16897,판도라상자(igno****),무한도전 쓰레기들로 인해 최악의 더빙 영화가 되었다 장난치려면 다른 쓰레기 영화 장...,1,13,27,2015-09-29 23:36:00,0
16898,16898,sung****,노래가 너무 좋았고 다 괜찮았음,1,1,18,2014-09-05 09:25:00,관람객


# 형태소 분석

In [104]:
# konlpy(한국어형태소분석기) 중에서 Okt 실행하기
from konlpy.tag import Okt
okt = Okt()

In [105]:
# "원본데이터NAN삭제"를 대상으로 "okt"를 사용해서 형태소 분석하기
저장공간  = []
for index, row in 원본데이터NAN삭제.iterrows(): # 원본데이터NAN삭제의 컬럼(행)을 불러와서 반복할 준비!
    대상텍스트 = row[2] # 분석에서 사용할 텍스트 정보가 있는 열을 지정해준다. 주의! python은 0부터 시작한다!
    ## 바로: 대상 텍스트의 열은 2번입니다. python은 0부터 숫자를 세게 됩니다!!!
    분류 = row[3] # 분석에서 사용할 분류 정보가 있는 열을 지정해준다. 주의! python은 0부터 시작한다!
    ## 바로: 평점 정보는 3번째 열에 있습니다.
    형태소 = list(okt.pos(대상텍스트)) # okt의 pos 기능을 활용해서 대상 텍스트를 형태소 분리한다.
    형태소종합 = pd.DataFrame.from_records(형태소) # 형태소에 저장된 내용을 데이터프레임 형태로 해서 형태소종합으로 저장한다.
    형태소종합['type'] = 분류 # 형태소종합에 분류정보를 추가한다.
    형태소종합['count'] = 1   # 형태소종합에 카운트정보를 추가한다.   
    저장공간.append(형태소종합) # 형태소종합의 내용을 종합하여 저장한다.

In [80]:
# "형태소" 변수의 저장 내용 확인
형태소

[('절대', 'Noun'),
 ('보지마', 'Verb'),
 ('.', 'Punctuation'),
 ('왕', 'Noun'),
 ('짜증', 'Noun'),
 ('.', 'Punctuation'),
 ('보다보다', 'Verb'),
 ('중건', 'Noun'),
 ('나옴', 'Verb'),
 ('.', 'Punctuation'),
 ('좋다는', 'Adjective'),
 ('의견', 'Noun'),
 ('은', 'Josa'),
 ('아마', 'Noun'),
 ('돈빋', 'Noun'),
 ('고', 'Josa'),
 ('알바생', 'Noun'),
 ('들', 'Suffix'),
 ('이', 'Josa'),
 ('댓글달았을듯', 'Verb'),
 ('.', 'Punctuation'),
 ('수입', 'Noun'),
 ('했으니', 'Verb'),
 ('본전', 'Noun'),
 ('은', 'Josa'),
 ('뽑아', 'Verb'),
 ('야할테고', 'Adjective'),
 ('.', 'Punctuation'),
 ('한국영', 'Noun'),
 ('화가', 'Noun'),
 ('최고다', 'Noun'),
 ('.', 'Punctuation'),
 ('.', 'Punctuation'),
 ('살다', 'Verb'),
 ('살다', 'Verb'),
 ('이렇게', 'Adverb'),
 ('재미없는', 'Adjective'),
 ('ㆍ', 'Foreign'),
 ('영화', 'Noun'),
 ('는', 'Josa'),
 ('.', 'Punctuation')]

In [107]:
# "형태소종합" 변수의 저장 내용 확인
형태소종합

,0,1,type,count
0,성우,Noun,1,1
1,들,Suffix,1,1
2,1년,Number,1,1
3,에,Foreign,1,1
4,한번,Noun,1,1
5,더빙,Noun,1,1
6,할,Verb,1,1
7,기회,Noun,1,1
8,생겼는데,Verb,1,1
9,그걸,Adverb,1,1


In [108]:
# "저장공간" 변수의 저장 내용 확인
저장공간

[      0               1  type  count
 0    박평            Noun    10      1
 1     식          Suffix    10      1
 2   평론가            Noun    10      1
 3     는            Josa    10      1
 4   그렇게          Adverb    10      1
 5   재밌는       Adjective    10      1
 6    영화            Noun    10      1
 7     가            Josa    10      1
 8   없으면       Adjective    10      1
 9    영화            Noun    10      1
 10    를            Josa    10      1
 11   무슨        Modifier    10      1
 12   재미            Noun    10      1
 13    로            Josa    10      1
 14   볼까            Verb    10      1
 15    ?     Punctuation    10      1
 16    ㅉ  KoreanParticle    10      1,         0            1  type  count
 0      평점         Noun    10      1
 1      처음         Noun    10      1
 2       써         Verb    10      1
 3      봐요         Verb    10      1
 4     ...  Punctuation    10      1
 5      진짜         Noun    10      1
 6       오     Modifier    10      1
 7     에스티         N

In [109]:
# "저장공간" 변수의 내용을 데이터프레임 형식으로 변환해서 "분석통합" 변수에 저장한다.
분석통합 = pd.concat(저장공간)

In [110]:
# "분석통합" 변수의 저장 내용 확인
분석통합

,0,1,type,count
0,박평,Noun,10,1
1,식,Suffix,10,1
2,평론가,Noun,10,1
3,는,Josa,10,1
4,그렇게,Adverb,10,1
...,...,...,...,...
17,찬양,Noun,1,1
18,하는,Verb,1,1
19,무도,Noun,1,1
20,충,Noun,1,1


In [112]:
# "분석통합" 변수의 컬럼(열) 이름 변경
분석통합.columns = ["형태소", "품사", "분류", "카운트"]

In [113]:
# "분석통합" 변수의 저장 내용 확인
분석통합

,형태소,품사,분류,카운트
0,박평,Noun,10,1
1,식,Suffix,10,1
2,평론가,Noun,10,1
3,는,Josa,10,1
4,그렇게,Adverb,10,1
...,...,...,...,...
17,찬양,Noun,1,1
18,하는,Verb,1,1
19,무도,Noun,1,1
20,충,Noun,1,1


In [114]:
# "분석통합" 변수의 내용을 형태소, 품사, 분류가 같은 것을 합치고, 카운트의 총합을 구해서 "그룹통합" 변수에 저장한다.
그룹통합 = 분석통합.groupby(['형태소', '품사', "분류"])['카운트'].sum()

In [115]:
# "그룹통합" 변수의 저장 내용 확인
그룹통합

형태소  품사           분류
    Foreign      10    1

    Foreign      1     1
!    Punctuation  1     6
                  3     1
                  4     1
                       ..
힙합   Noun         9     1
힝    Noun         10    1
힣    Exclamation  10    1
￣▽￣  Foreign      10    1
￦★   Foreign      10    1
Name: 카운트, Length: 36689, dtype: int64

In [116]:
# "그룹통합" 변수의 내용을 데이터프레임 형식으로 변환해서 "그룹통합" 변수에 저장한다.
그룹통합 = pd.DataFrame(그룹통합)

In [117]:
# "그룹통합" 변수의 저장 내용 확인
그룹통합

카운트
형태소 품사          분류     
   Foreign     10    1

   Foreign     1     1
!   Punctuation 1     6
                3     1
                4     1
...                 ...
힙합  Noun        9     1
힝   Noun        10    1
힣   Exclamation 10    1
￣▽￣ Foreign     10    1
￦★  Foreign     10    1

[36689 rows x 1 columns]

In [119]:
# "그룹통합" 변수의 내용을 "형태소분석결과.csv" 파일로 저장한다.
# header는 컬럼(열) 정보의 포함 여부이다.
# encoding은 문자코드를 선택하는 것이다. python에서는 기본적으로 utf-8(유니코드)를 사용한다.

그룹통합.to_csv('형태소분석결과.csv', header='true', encoding='utf-8')